# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:

weather_results_path = "weather_df.csv"
weather_results = pd.read_csv(weather_results_path)
weather_results.head()

,city_name,Date,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,albany,1596401825,US,42.60,-73.97,87.01,62,60,11.39
1,new norfolk,1596401825,AU,-42.78,147.06,45.00,82,78,1.99
2,puerto ayora,1596401825,EC,-0.74,-90.35,77.00,69,75,13.87
3,hirado,1596401826,JP,33.36,129.55,78.01,81,86,1.01
4,trairi,1596401826,BR,-3.28,-39.27,78.91,77,4,11.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
gmaps.configure(api_key=g_key)

location = weather_results[["Latitude", "Longitude"]]
humidity = weather_results["Humidity"].astype(float)


In [20]:
#Make a heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
index = weather_results[(weather_results["Humidity"] > 45) | \
                  (weather_results["Wind Speed"] > 25) | \
                  (weather_results["Temperature"] < 75) | \
                  (weather_results["Temperature"] > 90) | \
                  (weather_results["Cloudiness"] > 20)].index.tolist()
hotel_df = weather_results.drop(index)
hotel_df

,city_name,Date,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
27,corinto,1596401779,GR,37.94,22.96,79.00,44,0,1.01
29,whitefish,1596401669,US,48.41,-114.34,88.00,33,1,4.70
36,ravar,1596401830,IR,31.27,56.81,77.07,18,0,1.05
114,akdepe,1596401839,TM,42.06,59.38,80.60,44,0,4.70
122,thompson,1596401840,CA,55.74,-97.86,77.00,36,20,4.70
180,mehriz,1596401847,IR,31.59,54.43,89.60,21,20,4.70
187,caraquet,1596401848,CA,47.79,-64.94,86.00,40,1,8.05
200,palaikastron,1596401849,GR,35.20,26.25,82.99,43,0,14.00
222,barra do garcas,1596401852,BR,-15.89,-52.26,83.43,28,0,5.66
225,la ronge,1596401852,CA,55.10,-105.28,78.80,44,20,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
#Add hotel name column 
hotel_df["Hotel Name"] = ""


In [65]:
#Set parameters and search
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in hotel_df.iterrows():
    params = {
    "keyword": "Hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}
    params["location"] = f"{row['Latitude']},{row['Longitude']}"
    response = requests.get(url, params=params).json()
    
    results = response['results']
    hotel_df.head()
    

In [62]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]


In [60]:
# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))